In [3]:
import PIL
from PIL import Image
import imagehash

import pycamhd.pycamhd as camhd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xar
import urllib as url                     

def diff_letters(a,b):
    if (len(a) != len(b)):
        return -1
    else:
        return sum ( a[i] != b[i] for i in range(len(a)) )

In [2]:
files = camhd.get_file_list()
fileIDs = []
avgHash = []
dHash = []
pHash = []
wHash = []

In [ ]:
for filename in files:
    movAtom = camhd.get_moov_atom(filename)
    frameTotal = camhd.get_frame_count(movLink, movAtom) 
    frames = range(0,frameTotal,30)
    #frames every ~1 second
    
    for i in frames:
        ID = get_timestamp(movAtom) + i
        frame = camhd.get_frame(filename, i)
        im = Image.fromarray(frame)
        #avg = imagehash.average_hash(im)
        #p = imagehash.phash(im)
        #d = imagehash.dhash(im)
        w = imagehash.whash(im)
        
        #avgHash.append(avg)
        #pHash.append(p)
        #dHash.append(d)
        wHash.append(w)
        

In [ ]:
plt.rc('font', size=12)
fig, ax = plt.subplots();
fig.set_size_inches(14, 6);
fig.frameon = False
hb = ax.hexbin(timestamp, wHash, vmin=0, vmax=1, bins='log', linewidths=0.25,
  gridsize=(225,4500), mincnt=1, cmap=plt.cm.BuPu)
fig.colorbar(hb)
ax.set_ylim([0, 8000])
ax.set_xlim([timestamp[0],timestamp[-1]])
ax.yaxis.grid(True)
ax.xaxis.grid(True)